In [99]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf #needed for models in this script
import pylab as pl
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [100]:
pd.set_option('html', True) #see the dataframe in a more user friendly manner
%matplotlib inline

## Web Scraping Introduction

For a great look at web scraping, see this blogpost (https://thomaslevine.com/!/web-sites-to-data-tables/) by data scientist and civic hacker Thomas Levine.

His process for determining when to scrape is important:

1. I ponder whether it’s worth doing at all. I first want to know that someone will use the data table that I produce.

2. I ponder whether it’s worth automating nicely. If the data are just a few numbers spread across a few pages, it’s probably not worth writing a special thing.

3. I figure out how a person would navigate all of the websites and do whatever is desired.

4. I start automating just part of the human process. I try to start with a tiny part that is easy and will be useful quickly.

## HTML and CSS for Web Scraping

As we've seen in the previous lessons, in order to access the data we want, we have to know something about the target. In the case of downloading data from a website, it's simply knowing the URL string that takes us to the file location online. In the case of an API, it's knowing not only the service endpoint, but also the available query parameters and setting up the authentication tokens.

Often the data we wanted is locked away in tables on a webpage. To extract the data, we need to download the page and remove only those elements we want.

Fundamentally, HTML is just as it sounds: a markup language for text that tells a browser how to display (or render) the text. The markup is done with tags that are added to the text to indicate simple things like the font to use or the size of the text. They can also specify the location on the page where the text should appear. These are usually the type of tags we're interested in. In most cases, the data we want is embedded in a tag. An HTML table tag has a very simple structure:

![](files\UN_data1.jpg)

There are many tools for accessing HTML, which at its heart is just a text file. In the browser, there are web development tools that allow you to inspect the HTML code and explore the structure of the webpage.

In the following assignment, we're going to be scraping data from a webpage and process it in Python. While the example is comparatively basic, the task is the same no matter what webpage you're scraping data off of. Before you scape, make sure you read and abide by any terms of use posted on the site.

## Scrape Data from the UN

We're going to take a simple example. In this case, we're going to scrape school life expenctancy data for countries around the world. This is a measure of how long someone is likely to stay in school. The page we're querying comes from the Internet Archive. The current page is more sophisticated and allows users to download this data as an Excel file.

The first step is to inspect the page. Go to the link (http://web.archive.org/web/20110514112442/http://unstats.un.org/unsd/demographic/products/socind/education.htm) and check out the page with a web developer tool <b>(in Chrome and Firefox, right click on the page and select "Inspect Element").</b> This will show you the HTML powering the page.

If you click through the tags, you'll notice a number of tables on the page. In the beginning of the web, tables were used to arrange elements on a page. More recently, webpage designers use other methods to arrange content, but some pages (like this one) still use tables to keep the content properly arranged.

You can see the data we want here:

![](files/UN_data2.jpg)

To get the data we want, we need to extract the right table and get just the data in between the tags. In this case, we're going to use a package called BeautifulSoup and our old friend, Requests:

In [101]:
from bs4 import BeautifulSoup
import requests

Import the page as we've been doing previously:

In [102]:
url = "http://web.archive.org/web/20110514112442/http://unstats.un.org/unsd/demographic/products/socind/education.htm"
un_data = requests.get(url)

Then we pass the result to BeautifulSoup to do its thing:

In [103]:
un_soup = BeautifulSoup(un_data.content)

In [104]:
un_soup.head()

[<script src="/static/js/analytics.js" type="text/javascript"></script>,
 <script type="text/javascript">archive_analytics.values.server_name="wwwb-app7.us.archive.org";archive_analytics.values.server_ms=202;</script>,
 <link href="/static/css/banner-styles.css" rel="stylesheet" type="text/css"/>,
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 <title>United Nations Statistics Division - Demographic and Social Statistics</title>,
 <style type="text/css">
 <!--
 .style8 {font-size:10px; background-color:#fff; font-family: verdana, geneva, san-serif;}
 -->
 </style>,
 <link href="/web/20110514112442cs_/http://unstats.un.org/unsd/unsd2.css" rel="stylesheet" type="text/css"/>,
 <link href="/web/20110514112442cs_/http://unstats.un.org/unsd/unsd.css" rel="stylesheet" type="text/css"/>,
 <script language="javascript" src="/web/20110514112442js_/http://www.un.org/search/formValidate.js"></script>,
 <script language="javascript" src="/web/20110514112442js_/http://www.un.

As the name implies, the webpage content exists as a mess of text in the soup object. We need to extract the table we want, so we start trying to filter through, checking each table for the content we want:

In [112]:
for row in un_soup('table'):
    print row

<b>Note:</b> Above cell has no output shown; for clarity on Github.

<b>Comment:</b> The instructions say nothing about how we are supposed to know it's table 7 (index 6). My assumption is we are supposed to count manually the number of table entries until we arrive at the one we want. Is this correct? I tried this and could not arrive at the correct index. So I don't know how I'm supposed to know which table it is. In addition, if there were hundreds of tables this would be a tedious task prone to many errors. 

Finally we see the 7th table (at index 6) has the data we want:   

In [118]:
print (un_soup('table')[6])

<table border="0" cellpadding="0" cellspacing="0" width="800">
<tr valign="top">
<td><!-- #BeginEditable "MainContent" -->
<table align="center" border="0" width="540">
<tr>
<td cllcontsubhd="" colspan="2" valign="bottom"><h2 align="center"><a name="Top"></a>Social 
              indicators </h2></td>
</tr>
<tr>
<td cllcontsubhd="" colspan="2" height="23" valign="top"><div align="center" class="csubhd"><a href="default.htm">Introduction</a> 
              | <a href="statistics.htm">Statistics</a></div></td>
</tr>
</table>
<table cellpadding="0" cellspacing="0">
<tr>
<td height="1" width="797"><div align="right">
<p class="Clear"><a href="Education_Dec2010.xls" style="color: #666; font-weight: bold">Download data</a> <a href="Education_Dec2010.xls"><img alt="Excel" border="0" height="21" name="Excel" src="/web/20110514112442im_/http://unstats.un.org/unsd/img/excel.gif" width="22"/></a></p>
<div align="left">
<table align="left" cellpadding="0" cellspacing="0">
<tr class="bar1">
<td cols

Now we want to return all the tags. Look through the documentation (http://www.crummy.com/software/BeautifulSoup/bs4/doc/) and see if you can get the elements from the table. You'll still need to process the rows once you retrieve them and deal with the spaces and empty field separators. 

In [115]:
print (un_soup('table')[6].prettify())[0:1000] #limit output - remove if you want to see all
# The prettify() method will turn a Beautiful Soup parse tree into a nicely formatted Unicode string, 
# with each HTML/XML tag on its own line:

<table border="0" cellpadding="0" cellspacing="0" width="800">
 <tr valign="top">
  <td>
   <!-- #BeginEditable "MainContent" -->
   <table align="center" border="0" width="540">
    <tr>
     <td cllcontsubhd="" colspan="2" valign="bottom">
      <h2 align="center">
       <a name="Top">
       </a>
       Social 
              indicators
      </h2>
     </td>
    </tr>
    <tr>
     <td cllcontsubhd="" colspan="2" height="23" valign="top">
      <div align="center" class="csubhd">
       <a href="default.htm">
        Introduction
       </a>
       |
       <a href="statistics.htm">
        Statistics
       </a>
      </div>
     </td>
    </tr>
   </table>
   <table cellpadding="0" cellspacing="0">
    <tr>
     <td height="1" width="797">
      <div align="right">
       <p class="Clear">
        <a href="Education_Dec2010.xls" style="color: #666; font-weight: bold">
         Download data
        </a>
        <a href="Education_Dec2010.xls">
         <img alt="Excel" border="0


We want to find all the tables in this 'table', so we can search for < tr > and the tables we are interested in all have class = "tcont", so we will search for these:

In [38]:
un_step1 = un_soup('table')[6].find_all('tr', {'class': 'tcont'})
un_step1[:1]

[<tr class="tcont">
 <td height="19">Afghanistan</td>
 <td align="right" height="19">2004</td>
 <td height="19"> </td>
 <td height="19">a</td>
 <td align="right" height="19" width="71">8</td>
 <td width="6"></td>
 <td height="19" width="51"></td>
 <td align="right" height="19" width="72">11</td>
 <td width="6"></td>
 <td height="19" width="53"></td>
 <td align="right" height="19" width="64">5</td>
 <td height="19" width="59"></td>
 </tr>]

<b>Comment:</b> I'm not sure why I had to use {} to make the 'class' work. I tried () but that failed. I then just guessed.

<b>ADDITIONAL COMMENT: using '.find_all('tr', {'class':'tcont'}) does NOT return all of the countries. It appears only about half of the countries have the class = tcont before the country name, the other country names have nothing (ex. Sweden) so this code is not picking up those countries. I have no idea how I'm supposed to search for those countries at this point so I'm stuck here.</b>

In [39]:
len(un_step1)

99

This would inidicate we have 99 countries worth of data. 

We have to take each 'table' and pull out the data required to populate our database:   

1. Country
2. Year data is from
3. Total years of education
4. Total years for Men's education
5. Total years for Women's education

The below blocks of code will test one table, once it's working correctly we'll implement it to work for all tables:

In [40]:
#take one table:
testing1 = un_step1[:1]
testing1

[<tr class="tcont">
 <td height="19">Afghanistan</td>
 <td align="right" height="19">2004</td>
 <td height="19"> </td>
 <td height="19">a</td>
 <td align="right" height="19" width="71">8</td>
 <td width="6"></td>
 <td height="19" width="51"></td>
 <td align="right" height="19" width="72">11</td>
 <td width="6"></td>
 <td height="19" width="53"></td>
 <td align="right" height="19" width="64">5</td>
 <td height="19" width="59"></td>
 </tr>]

In [62]:
# get all the lines individually, exlcuding <tr> lines:
for rows in testing1:
    col = rows.find_all('td')
    print col

[<td height="19">Afghanistan</td>, <td align="right" height="19">2004</td>, <td height="19"> </td>, <td height="19">a</td>, <td align="right" height="19" width="71">8</td>, <td width="6"></td>, <td height="19" width="51"></td>, <td align="right" height="19" width="72">11</td>, <td width="6"></td>, <td height="19" width="53"></td>, <td align="right" height="19" width="64">5</td>, <td height="19" width="59"></td>]


In [63]:
#see what each line prints out as a string:
print 'line 0:', col[0].string
print 'line 1:', col[1].string
print 'line 2:', col[2].string
print 'line 3:', col[3].string
print 'line 4:', col[4].string
print 'line 5:', col[5].string
print 'line 6:', col[6].string
print 'line 7:', col[7].string
print 'line 8:', col[8].string
print 'line 9:', col[9].string
print 'line 10:', col[10].string
print 'line 11:', col[11].string

line 0: Afghanistan
line 1: 2004
line 2:  
line 3: a
line 4: 8
line 5: None
line 6: None
line 7: 11
line 8: None
line 9: None
line 10: 5
line 11: None


From the above, we see which lines have the information we require:

1. Index 0 = 'country'
2. Index 1 = 'year'
3. Index 4 = 'total years'
4. Index 7 = 'men years'
5. Index 10 = 'women years'

Now we can write code that will return a dataframe with the data for all countries:

In [89]:
holding_frame = [] #empty list to hold data

for rows in un_step1:
    col = rows.find_all('td')
    country = col[0].string
    year = col[1].string
    total = col[4].string
    men = col[7].string
    women = col[10].string
    hold_tuple = (country, year, total, men, women) #tuple so when data is appened it stays in the proper order
    holding_frame.append(hold_tuple)

column_names = ['country', 'year', 'total_school_years', 'total_men_years', 'total_women_years']
UN_school_frame = pd.DataFrame(holding_frame, columns = column_names)

UN_school_frame.head()

IndexError: list index out of range

In [90]:
holding_frame

[(u'Afghanistan', u'2004', u'8', u'11', u'5'),
 (u'Albania', u'2004', u'11', u'11', u'11'),
 (u'Algeria', u'2005', u'13', u'13', u'13'),
 (u'Andorra', u'2008', u'11', u'11', u'12'),
 (u'Anguilla', u'2008', u'11', u'11', u'11'),
 (u'Austria', u'2008', u'15', u'15', u'15'),
 (u'Azerbaijan', u'2008', u'13', u'13', u'13'),
 (u'Bahrain', u'2006', u'14', u'14', u'15'),
 (u'Bangladesh', u'2007', u'8', u'8', u'8'),
 (u'Belarus', u'2007', u'15', u'14', u'15'),
 (u'Bolivia (Plurinational State of)', u'2007', u'14', u'14', u'14'),
 (u'Bosnia and Herzegovina', u'2009', u'14', u'13', u'14'),
 (u'Botswana', u'2006', u'12', u'12', u'12'),
 (u'Brazil', u'2008', u'14', u'14', u'14'),
 (u'British Virgin Islands', u'2005', u'17', u'15', u'19'),
 (u'Cameroon', u'2009', u'10', u'11', u'9'),
 (u'Canada', u'2002', u'16', u'16', u'16'),
 (u'Cape Verde', u'2009', u'12', u'11', u'12'),
 (u'Cayman Islands', u'2008', u'12', u'11', u'12'),
 (u'Central African Republic', u'2009', u'7', u'8', u'5'),
 (u'Colombia', u